In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from fpdf import FPDF
from bs4 import BeautifulSoup
import time
import os
import html
import warnings
warnings.filterwarnings("ignore", category=UserWarning)  # 경고 무시

# 브라우저 드라이버 설정
chrome_options = Options()
# 다운로드 경로 설정
save_directory = "pdf_files"
if not os.path.exists(save_directory):
    os.makedirs(save_directory)

chrome_options.add_argument("--headless")  # 헤드리스 모드
chrome_options.add_experimental_option("prefs", {
    "download.default_directory": os.path.abspath(save_directory),  # 다운로드 경로
    "download.prompt_for_download": False,  # 다운로드 시 팝업을 뜨지 않게 설정
    "plugins.always_open_pdf_externally": True  # PDF 파일 자동으로 외부에서 열기
})
driver = webdriver.Chrome(options=chrome_options)

# 첫 페이지 URL
base_url = "https://xn--vb0b6f546cmsg6pn.com/sub/preced/preced01.asp?mode=list&bid=8&s_type=&s_keyword=&s_cate=&page="

# 총 6페이지 반복
for page in range(1, 7):
    # 현재 페이지 열기
    url = f"{base_url}{page}"
    driver.get(url)

    # 페이지가 로드될 때까지 기다리기
    WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.XPATH, '//tbody/tr/td[@class="left"]/a')))

    # 모든 링크 가져오기
    links = driver.find_elements(
        By.XPATH, '//table[@class="ntb-tb"]/tbody/tr/td[@class="left"]/a')

    # 각 링크 처리
    for link in links:
        href = link.get_attribute("href")
        if href:
            # 링크로 이동
            driver.get(href)
            time.sleep(2)  # 페이지 로드 대기

            try:
                # PDF 링크 추출 (tbody 안의 세 번째 tr 안의 td class='left' 내 a href)
                pdf_link_element = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located(
                        (By.XPATH, '//table[@class="ntb-tb-view"]/tbody/tr[4]/td[@class="left"]/a'))
                )
                pdf_url = pdf_link_element.get_attribute("href")

                # PDF 다운로드ㅇ
                if pdf_url.endswith(".pdf"):
                    driver.get(pdf_url)  # PDF 링크 열기

            except Exception as e:
                pass

                # PDF 링크를 찾을 수 없으면 content-area에서 텍스트 추출
                try:
                    # content-area의 div 요소 가져오기
                    content_area = WebDriverWait(driver, 10).until(
                        EC.presence_of_element_located(
                            (By.XPATH,
                             '//table[@class="ntb-tb-view"]/tbody/tr[3]/td/div[@class="content-area"]')
                        )
                    )

                    # HTML 내용 추출
                    html_content = content_area.get_attribute("outerHTML")

                    # BeautifulSoup을 사용하여 HTML 태그 제거
                    soup = BeautifulSoup(html_content, 'html.parser')

                    # HTML 엔티티를 공백으로 변환하지 않도록 설정
                    text = soup.get_text(separator="")  # separator를 빈 문자열로 설정

                    # 줄 바꿈을 원하는 부분에만 추가 (예: 각 p 태그 사이에만 줄 바꿈 추가)
                    paragraphs = soup.find_all('p')
                    formatted_text = ""

                    for paragraph in paragraphs:
                        paragraph_text = paragraph.get_text().strip()
                        if paragraph_text:
                            formatted_text += paragraph_text + "\n"  # 각 p 태그마다 줄 바꿈 추가

                    # 'nbsp' 엔티티를 실제 공백으로 바꾸기
                    # '\xa0'은 &nbsp;에 해당하는 공백입니다.
                    formatted_text = formatted_text.replace('\xa0', ' ')

                    # PDF 생성
                    class PDF(FPDF):
                        def __init__(self):
                            super().__init__()
                            # 한글 폰트 추가
                            self.add_font('NanumGothic', '',
                                          'NanumGothic.ttf', uni=True)

                    # 한글 폰트 파일 확인
                    font_path = "NanumGothic.ttf"
                    if not os.path.exists(font_path):
                        raise FileNotFoundError(
                            "한글 폰트 파일(NanumGothic.ttf)을 다운로드하여 현재 디렉토리에 저장하세요."
                        )

                    # PDF 파일 저장 경로
                    save_directory = "pdf_files"  # 저장할 폴더 이름
                    file_name = os.path.join(
                        save_directory, f"page_{page}_file_{links.index(link) + 1}.pdf")

                    # PDF 객체 생성
                    pdf = PDF()
                    pdf.add_page()
                    pdf.set_font('NanumGothic', '', 12)

                    # 줄 높이를 6으로 설정하고 텍스트 저장
                    pdf.multi_cell(0, 6, formatted_text)

                    # PDF 저장
                    pdf.output(file_name)

                except Exception as ex:
                    pass

            finally:
                # 원래 페이지로 돌아가기 (예외 발생 여부와 관계없이 실행)
                driver.back()
                time.sleep(2)

# 크롤링 종료
driver.quit()
print("크롤링 완료 및 PDF 저장 완료")

KeyboardInterrupt: 